In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Additional libraries added for project:

import matplotlib.pyplot as plt
import seaborn as sns

#Data processing
from sklearn.impute import KNNImputer

#Model selection
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel

#ML algorthms
from xgboost import XGBRegressor

#Deep learning
import tensorflow as tf

#other

from IPython.display import Image, display

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

<h2>Project Scope</h2>

   <ul>
            <li>Using solar generation and associated temperature data to explore time series forecasting and make a short-term solar power forecasting model.</li>
            <li>Examine and process outliers/missing data</li>
            <li>Compare the performance of an XGBoost model vs a deep learning LSTM model to make day-ahead forecasts</li>
            <li>See what impact including the temperature data has on the forecast</li>
            <li>Experiment with feature engineering and test how differernt features effect the model's performance</li>
            <li>Make a full pipeline of the best model.</li> 
            <li>A better way?</li>
            <li>Report conclusions and comment on possible project extensions</li>
   </ul>

<h2>Loading and exploring data</h2>
This data was taken from two solar plants in India over the course of 34 days in 15 minute intervals. The weather data is on the plant level (just measured from one sensor) and the generation data is gathered from individual inverters across the plant. More information can be found here: 
<a href="https://www.kaggle.com/datasets/anikannal/solar-power-generation-data">Data Card</a>. Credit to 
<a href="https://www.kaggle.com/anikannal">Ani Kannal</a> for uploading this dataset to Kaggle. 

### Generation data
For now we'll just look at the Plant 1 data.

In [2]:
df_plt1_gen = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv')

df_plt1_gen.sample(10, random_state=1)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv'

PLANT_ID is the same throughout, so it can be safely removed. The generation is 0 for nighttime, which of course makes sense for solar power. Also, let's rename to inverters/inverter column to make them easier to track

In [ ]:
#Since we're focusing on plant 1 for now we'll shorten the name
df_gen = df_plt1_gen.drop('PLANT_ID', axis=1)

df_gen['INVERTER'] = df_gen.SOURCE_KEY.map({df_gen.SOURCE_KEY.unique()[i-1]: f'INVERTER_{i}' for i in range(1, len(df_gen.SOURCE_KEY.unique()) +1)})
df_gen = df_gen.drop('SOURCE_KEY', axis=1)
inverters = df_gen.INVERTER.unique()
inverters

In [ ]:
df_gen.info()

In [ ]:
df_gen.describe(include='all').T

In [ ]:
df_gen.groupby('INVERTER').count().DATE_TIME.sort_values()

The dtypes make sense, but we'll need to make the DATE_TIME a datetime object for easier analysis. No nulls detected, but it looks like there could be missing data either at the plant or inverter level. At the plant level we would expect 3264 unique timestamps (34 days * 24 hours * 4 (15 minute intervals in hour)), but there are only 3158. Across all inverters there would be 71808 rows (3264 * 22 (inverters)), but we only have 68778. So overall, around 3000 or ~4% of the total expected are missing. There is also some variance per inverter for what datetimes are missing.

In [ ]:
df_gen['DATE_TIME'] = pd.to_datetime(df_gen.DATE_TIME, format='%d-%m-%Y %H:%M')
df_gen.dtypes

### Weather sensor data

In [ ]:
df_plt1_weather = pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')

df_plt1_weather.sample(10, random_state=1)

In [ ]:
df_plt1_weather.info()

In [ ]:
df_plt1_weather.describe(include='all').T

We can remove the PLANT_ID and SOURCE_KEY since they are the same throughout. Let's again make DATE_TIME a datetime object. Based on the DATE_TIME count we also have some missing weather data. 

In [ ]:
df_weather = df_plt1_weather.drop(['PLANT_ID', 'SOURCE_KEY'], axis=1)
df_weather['DATE_TIME'] = pd.to_datetime(df_weather.DATE_TIME, format='%Y-%m-%d %H:%M:%S') 

### Filling out missing datetimes
In order to fill in the missing datetime stamps for our generation and weather sensor dataframes let's make a datetime object with the full range of expected datetimes.

In [ ]:
datetimes_full = pd.Series(pd.date_range(df_gen.DATE_TIME.min(), df_gen.DATE_TIME.max(), freq='15min'), name='DATE_TIME')
datetimes_full

Now we are ready to merge this will our dataframes. Starting with the generation data. Since different inverters have some different timestamps missing we will need to do this one inverter at a time and then concatenate them back together.

In [ ]:
dfs_gen = []
for i in inverters:
    df_inverter = df_gen[df_gen.INVERTER == i]
    df_inverter = df_inverter.merge(datetimes_full, on='DATE_TIME', how='right')
    df_inverter['INVERTER'] = i
    dfs_gen.append(df_inverter)
df_gen = pd.concat(dfs_gen)
df_gen.info()

Now we can add the weather data to this as well to make our full dataframe.

In [ ]:
df = df_gen.merge(df_weather, on='DATE_TIME', how='left')
df.info()

## Exploratory Data Analysis
To start our more in-depth analysis, we'll can make some basic datetime features. If we were modeling throughout the year(s), month and year could be interesting to account for seasonal variation and long-term trends, but since our data only covers 34 days, we will omit them. We can use dayofyear to capture any longer trends that might be present.

In [ ]:
df['HOUR'] = df.DATE_TIME.dt.hour
df['DAY'] = df.DATE_TIME.dt.dayofyear
df['DAY_WEEK'] = df.DATE_TIME.dt.dayofweek
df['MINUTES_15'] = df.DATE_TIME.dt.time

#This maps the 15 minute intervals over the course of the day to ints 1-96. 
df['MINUTES_15'] = df.MINUTES_15.map({df.MINUTES_15.unique()[i-1]:i for i in range(1, 97)})
#Change day of year to day of data
df['DAY'] = df.DAY.map({df.DAY.unique()[i-1]:i for i in range(1, 35)})

Let's check on the linear correlation of the features.

In [ ]:
corr = df.corr(numeric_only=True)
corr

And let's make it a heatmap.

In [ ]:
sns.heatmap(corr);

Not surprisingly DC and AC power are highly correlated. This is good! It probably means the inverters in the plant are working correctly to convert the DC to AC. We will ultimately make AC_POWER our target. There is a strong correlation between HOUR and DAILY_YIELD, which makes sense as the daily yield increases throughout the day. For the weather sensor data there is not surprisingly a strong correlation between IRRADIATION and AC_POWER. Also between MODULE_TEMPERATURE and AC_POWER. Now we will look at a pairplot to see another representation of these relationships and look for any non-linear correlations.

In [ ]:
# g = sns.PairGrid(df.sample(10000, random_state=1), diag_sharey=False, hue='INVERTER')
# g.map_upper(sns.scatterplot, s=15)
# g.map_lower(sns.kdeplot)
# g.map_diag(sns.histplot)
# #This takes quite a while to run, so saving this figure for future use.
# plt.savefig("gen_pairgrid.png", dpi=400)
# display(Image(filename='gen_pairgrid.png'))

There is A LOT of information to take in here! The zeros dominate the distribution for AC_POWER and DAILY_YIELD due to the nighttime. As we just saw perviously, there is further evidence of the missing data based on the DAY/MINUTES_15 pair. The inverters start to seperate out based on TOTAL_YIELD and DAILY_YIELD. This suggests the they are operating a different capacities; probably due to any number of factors: age, location within the plant, need for maintaince, etc. Exploring this more is outside the scope of this project, but could be a good starting point for another project. Finally, the connection between HOUR and AC_POWER is interesting and also makes sense with the peaks in the middle of the day. Let's explore this more next and also start to look for outliers.

## Outliers
First a quick check for any negative power numbers.

In [ ]:
df[['AC_POWER', 'DC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']].any() < 0

They should only be positive and luckily that appears to be the case.

### Target outliers
Let's now explore our target, AC_POWER. We can define an outlier as \< 1th percentile or \> 99th percentile. To do this I will make four quantile features below: 2 of them in the outlier range and 2 to establish 1 standard deviation from the mean. Then we can plot the results. 

In [ ]:
#Making a new dataframe for EDA since we won't necessarily want these features for our model.
df_eda = df.copy()
df_eda = df_eda.merge(df_eda.groupby('MINUTES_15').quantile(0.01, numeric_only=True).AC_POWER.rename('OUTLIERS_LOW_AC_POWER'), on='MINUTES_15', how='left')
df_eda = df_eda.merge(df_eda.groupby('MINUTES_15').quantile(0.99, numeric_only=True).AC_POWER.rename('OUTLIERS_HIGH_AC_POWER'), on='MINUTES_15', how='left')
df_eda = df_eda.merge(df_eda.groupby('MINUTES_15').mean(numeric_only=True).AC_POWER.rename('MEAN'), on='MINUTES_15', how='left')
df_eda = df_eda.merge(df_eda.groupby('MINUTES_15').std(numeric_only=True).AC_POWER.rename('STD'), on='MINUTES_15', how='left')
df_eda['STD_1'] = df_eda.query('AC_POWER < (MEAN + STD) and AC_POWER > (MEAN - STD)').AC_POWER

In [ ]:
#Since we have nulls in our data now we will exclude those with df[~df.isna()]
fig, ax = plt.subplots()
sns.color_palette("Paired")
sns.scatterplot(data=df_eda[~df_eda.isna()], y='AC_POWER', x='MINUTES_15', hue='INVERTER', palette='gray', alpha=0.05, legend=False)
sns.scatterplot(data=df_eda[~df_eda.isna()], y='STD_1', x='MINUTES_15', hue='INVERTER', palette='gray', alpha =0.5, legend=False)
sns.scatterplot(data=df_eda[~df_eda.isna()].query('AC_POWER > OUTLIERS_HIGH_AC_POWER'), y='AC_POWER', x='MINUTES_15', hue='INVERTER', size='AC_POWER')
sns.scatterplot(data=df_eda[~df_eda.isna()].query('AC_POWER < OUTLIERS_LOW_AC_POWER'), y='AC_POWER', x='MINUTES_15', hue='INVERTER', size= -df_eda[~df_eda.isna()].AC_POWER) 

plt.title('AC Power per 15 minutes')
plt.ylabel('AC Power (kW)')
plt.xlabel('Hour of Day')
ax.set_xticks([i for i in range(1, 97, 4)])
ax.set_xticklabels([i for i in range(24)])
ax.legend(['Full data', '+/- 1\u03C3', 'Outliers']);

Interestingly, it looks like there are several instances where the power generated during the middle of the day was 0. This could either indicate bad data, malfunctioning invererters, some kind of planned maintence, really cloudy days, really sunny days where the plant is overheating/at capacity or any number of other things. Without having a deeper domain knowledge it is hard to know for sure, but let's see if we can find any pattern to these mid-day outliers.

First, we'll check to see if any inverters in particular are responsible.

In [ ]:
outliers_low = df_eda[~df_eda.isna()].query('AC_POWER < OUTLIERS_LOW_AC_POWER')
outliers_high = df_eda[~df_eda.isna()].query('AC_POWER > OUTLIERS_HIGH_AC_POWER')

outliers_source_low = outliers_low.groupby('INVERTER').count().DATE_TIME.reset_index().sort_values('DATE_TIME', ascending=False)
outliers_source_low_zero = outliers_low[outliers_low.AC_POWER == 0].groupby('INVERTER').count().DATE_TIME.reset_index().sort_values('DATE_TIME', ascending=False)
fig, ax = plt.subplots()
sns.barplot(data = outliers_source_low, y='INVERTER', x='DATE_TIME', alpha = 0.5, )
sns.barplot(data = outliers_source_low_zero, y='INVERTER', x='DATE_TIME')
plt.title('Outlier Counts by Inverter (shading for 0.0 AC Power)')
plt.ylabel('Inverter')
plt.xlabel('Count');

The spread of outlier counts, including the 0.0 AC_POWER measurements are concentrated in inverters 11 and 1. It could be that these were offline for an extended period due to maintance or malfunction. Below we'll look at the spread of the outliers over the course of the full 34 days to see if there is any regularity.

In [ ]:
outliers_zero_count = outliers_low[outliers_low.AC_POWER == 0].groupby(['INVERTER','DAY']).count().rename(columns={'AC_POWER':'COUNTS'}).COUNTS.reset_index()
outliers_zero_count
fig, ax = plt.subplots(figsize=(8,5))
sns.scatterplot(data=outliers_zero_count, x='DAY', y='COUNTS', hue='INVERTER', size='COUNTS', legend='brief')
ax.legend(bbox_to_anchor=(1, 1.05))
ax.set_xticks([i for i in range(1,35)]);
ax.set_xticklabels([]);

Several occur on the same day. There could have been maintenance that day or another issue. Now let's check for weekly patterns that could suggest planned maintenance.

In [ ]:
outliers_zero_count = outliers_low[outliers_low.AC_POWER == 0].groupby(['INVERTER','DAY_WEEK']).count().rename(columns={'AC_POWER':'COUNTS'}).COUNTS.reset_index()
outliers_zero_count
fig, ax = plt.subplots(figsize=(8,2.5))
sns.scatterplot(data=outliers_zero_count, x='DAY_WEEK', y='COUNTS', hue='INVERTER', size='COUNTS', legend=False);

Interestingly, several of the zeros occur on Sunday. This could still be coincidence or it might suggest some scheduled maintance. Since Sunday is likely a day with less demand on the grid, it would make sense to choose that as a day for maintance or some planned outage. Finally, lets see how these outliers relate to the irradiation. distribution, since the two should be highly correlated. Let's circle back and check on the non-zero outliers as well.

In [ ]:
outliers_low_count = outliers_low.groupby(['INVERTER','DAY_WEEK']).count().rename(columns={'AC_POWER':'COUNTS'}).COUNTS.reset_index()
fig, ax = plt.subplots(figsize=(8,2.5))
sns.scatterplot(data=outliers_low_count, x='DAY_WEEK', y='COUNTS', hue='INVERTER', size='COUNTS', legend=False);

There are a greater number of overall outliers on Monday. We might guess that after Sunday maintance there is a delay getting everything back online that goes into Monday. I think there is enough evidence now to choose to keep these outliers in our data as it might help the model account for some of this. That being said the number is low enough that it likely won't have too much of an impact.

### Other Outliers
Let's take a quick look at outliers in the other features, before moving to missing data. Since we'll be exploring several different features we can make a function to help streamline the process.

In [ ]:
def outliers(df, feature, high_per, low_per, source_key=True):
    df = df.copy()
    df = df.merge(df.groupby('MINUTES_15').quantile(low_per, numeric_only=True)[feature].rename(f'OUTLIERS_LOW_{feature}'), on='MINUTES_15', how='left')
    df = df.merge(df.groupby('MINUTES_15').quantile(high_per, numeric_only=True)[feature].rename(f'OUTLIERS_HIGH_{feature}'), on='MINUTES_15', how='left')
    df = df.merge(df.groupby('MINUTES_15').mean(numeric_only=True)[feature].rename(f'MEAN_{feature}'), on='MINUTES_15', how='left')
    df = df.merge(df.groupby('MINUTES_15').std(numeric_only=True)[feature].rename(f'STD_{feature}'), on='MINUTES_15', how='left')
    df[f'STD_1_{feature}'] = df.query(f'{feature} < (MEAN_{feature} + STD_{feature}) and {feature} > (MEAN_{feature} - STD_{feature})')[feature]
    
    fig, ax = plt.subplots()
    sns.color_palette("Paired")
    if source_key == True:
        sns.scatterplot(data=df, y=feature, x='MINUTES_15', hue='INVERTER', palette='gray', alpha=0.05, legend=False)
        sns.scatterplot(data=df, y=f'STD_1_{feature}', x='MINUTES_15', hue='INVERTER', palette='gray', alpha =0.5, legend=False)
        sns.scatterplot(data=df.query(f'{feature} > OUTLIERS_HIGH_{feature}'), y=feature, x='MINUTES_15', hue='INVERTER', size=feature)
        sns.scatterplot(data=df.query(f'{feature} < OUTLIERS_LOW_{feature}'), y=feature, x='MINUTES_15', hue='INVERTER', size= -df[feature]) 
    else:
        sns.scatterplot(data=df, y=feature, x='MINUTES_15', color='gray', alpha=0.05, legend=False)
        sns.scatterplot(data=df, y=f'STD_1_{feature}', x='MINUTES_15', color='gray', alpha =0.5, legend=False)
        sns.scatterplot(data=df.query(f'{feature} > OUTLIERS_HIGH_{feature}'), color='blue', y=feature, x='MINUTES_15', size=feature)
        sns.scatterplot(data=df.query(f'{feature} < OUTLIERS_LOW_{feature}'), color='red', y=feature, x='MINUTES_15', size= -df[feature])
        
    plt.title(f'{feature} per 15 minutes')
    plt.ylabel(f'{feature}')
    plt.xlabel('Hour of Day')
    ax.set_xticks([i for i in range(1, 97, 4)])
    ax.set_xticklabels([i for i in range(24)])
    ax.legend(['Full data', '+/- 1\u03C3', 'Outliers']);
    return ax, df

In [ ]:
ax_daily_yield, df_daily_yield = outliers(df_eda[~df_eda.isna()], 'DAILY_YIELD', 0.99, 0.01);

There are some unusual things happening here. Mainly the 0s after 6pm. The straight line down at the end of the day and the other line at the start. These seem like they could be incorrect data, or maybe some kind of correction of previous data. Whatever their exact cause, it is probably best to correct these to make the data less disconnected and ultimately help our model. To fix these we'll make every value during the night equal to the maximum daily yield and set all the daily yields at the beginning of the day to 0.

In [ ]:
#Had a go through a few trys to settle on MINUTES_15 = 75 as a good starting point for night.
nighttime = [i for i in range(75,97)]
#Making these changes to the original df
df['DAILY_YIELD'] = df['DAILY_YIELD'].mask(df.HOUR.isin([0]), 0)
#We need a maximun daily yield for each inverter seperately
daily_yield = df.copy()
daily_yield = df.merge(pd.DataFrame({'DAILY_YIELD_DAY_MAX' : df.groupby([ 'INVERTER', 'DAY']).max()['DAILY_YIELD']}).reset_index(), on=['INVERTER', 'DAY'], how='right')
df['DAILY_YIELD'] = df['DAILY_YIELD'].mask(df.MINUTES_15.isin(nighttime), daily_yield['DAILY_YIELD_DAY_MAX'])

#Let's also set the nighttime generation for AC_POWER and DC_POWER to 0.
df['AC_POWER'] = df['AC_POWER'].mask(df.MINUTES_15.isin(nighttime), 0)
df['DC_POWER'] = df['DC_POWER'].mask(df.MINUTES_15.isin(nighttime), 0)

ax_daily_yield, df_daily_yield = outliers(df[~df.isna()], 'DAILY_YIELD', 0.99, 0.01);

This is better now. It remains to be seen how much of an inpact daily yield has on our model, but this should help at least a little.

## Imputing missing values
Now we are ready to impute the missing values as a final step before beginning our modeling. First, let's find a section of missing data to use as a test case of how effective our imputer is.

In [ ]:
sns.scatterplot(data=df[~df.isna()], x='MINUTES_15', y='DAY', hue='INVERTER', legend=False)
plt.title('Data points');

There is a fair bit of missing day time data on day 6, so we can use this day to test our imputer with a before and after plot

In [ ]:
sns.scatterplot(data=df[~df.isna()][df.DAY == 6], x='MINUTES_15', y='AC_POWER', hue='INVERTER', legend=False);
plt.title('Day 6 AC Power before imputing');

Let's begin imputing by setting all nighttime AC power generation to 0.

In [ ]:
df['AC_POWER'] = df['AC_POWER'].mask(df.HOUR.isin([0]), 0)


Now we can impute the other missing values. Let's try a couple of different methods and see what is more effective. We'll pick sklearn KNNImputer and also, based on this <a href="https://drnesr.medium.com/filling-gaps-of-a-time-series-using-python-d4bfddd8c460">article</a>, the built-in pandas method interpolate, with the time method specially. For best results we will apply our imputer to each inverter individually and then concatenate the results.

In [ ]:
imputer = KNNImputer(n_neighbors=10)
inverter_dfs = []
inverter_df = None
for i in inverters:
    inverter_df = df[df.INVERTER == i]
    inverter_df.pop('INVERTER')
    inverter_df.index = inverter_df['DATE_TIME']
    datetime = inverter_df.pop('DATE_TIME')
    inverter_df = pd.DataFrame(imputer.fit_transform(inverter_df), columns=inverter_df.columns, index=datetime)
    inverter_df['INVERTER'] = i
    inverter_df = inverter_df.reset_index()
    inverter_dfs.append(inverter_df)
df_knn = pd.concat(inverter_dfs)

sns.scatterplot(data=df_knn[df_knn.DAY == 6], x='MINUTES_15', y='AC_POWER', hue='INVERTER', legend=False);
plt.title('Day 6 AC Power Imputed with KNN');

In [ ]:
inverter_dfs = []
inverter_df = None
for i in inverters:
    inverter_df = df[df.INVERTER == i]
    inverter_df.pop('INVERTER')
    inverter_df.index = inverter_df['DATE_TIME']
    datetime = inverter_df.pop('DATE_TIME')
    inverter_df = inverter_df.interpolate(method='time')
    inverter_df['INVERTER'] = i
    inverter_df = inverter_df.reset_index()
    inverter_dfs.append(inverter_df)
df_interpolate = pd.concat(inverter_dfs)
sns.scatterplot(data=df_interpolate[df_interpolate.DAY == 6], x='MINUTES_15', y='AC_POWER', hue='INVERTER', legend=False);

plt.title('Day 6 AC Power Imputed with interpolate(time method)');

Both imputation methods seem to work pretty well, however, the KNNImputer appears to capture a bit more of the nuance of the data though, so we will choose that. Let's also make an imputer function.

In [ ]:
def imputer(df, imputer=KNNImputer(n_neighbors=10)):
    imputer = imputer
    inverter_dfs = []
    inverter_df = None
    for i in inverters:
        inverter_df = df[df.INVERTER == i]
        inverter_df.pop('INVERTER')
        inverter_df.index = inverter_df['DATE_TIME']
        datetime = inverter_df.pop('DATE_TIME')
        inverter_df = pd.DataFrame(imputer.fit_transform(inverter_df), columns=inverter_df.columns, index=datetime)
        inverter_df['INVERTER'] = i
        inverter_df = inverter_df.reset_index()
        inverter_dfs.append(inverter_df)
    df = pd.concat(inverter_dfs)
    return df

df = imputer(df)
df.info()

<!-- ## How to handle inverters? / Weather impact
So far we have had all of the inverters in one column of our dataframe with all of the their individual generation data in additional rows. For modeling though we should aggregate our dataframe to one datatime and make that our index. This leaves the question of what to do with the inverters. We could: aggregate accross them, greatly simplifying our dataframe, but possibly losing helpful information; or pivot them as new features to our dataframe making it much more complex; model each inverter seperately and make our model the sum of their predicitons. Also, as per our project scope, let's use this chance to see how the weather data impacts the models performance. We'll setup for each scenerio below -->

## Target

Before establishing a baseline we need to create our target. As mentioned previously our target will be the sum of the AC power generation between all the inverters for a given 15 minute interval forecasted ahead to the next day. So let's make the target and examine it a bit more closely before establishing a baseline.

In [ ]:
day = 96
#shifting to day ahead and droping nulls cause by the shift
target = df.groupby('DATE_TIME').sum(numeric_only=True).AC_POWER.shift(-day)
target = target.dropna()

Looking at our target over one week.

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.lineplot(target[target.index > '2020-06-10'])
plt.title('One Week of Power Generation (kW)')
plt.xlabel('Date')
plt.ylabel('kW');

This is what we will be modeling. There are a lot valleys and sharp peaks. Modeling the inverters seperately or including all the inverter generation data in the model could help address this.

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
sns.lineplot(df[(df.INVERTER == inverters[5]) & (df.DATE_TIME > '2020-06-10')].AC_POWER)
plt.title('Inverter 5: One Week of Power Generation (kW)')
plt.xlabel('Date')
plt.ylabel('kW');

However, looking at just one inverter actually doesn't seem to smooth out the data any, so the difference between including the inverter data seperately or just aggregating might be neglible. Either way, we'll explore both to compare the results. Let's look at the houlry results.

In [ ]:
target_hour = target.resample('H').sum()

fig, ax = plt.subplots(figsize=(15,5))
sns.lineplot(target_hour[target_hour.index > '2020-06-10'])
plt.title('One Week of Hourly Power Generation (kW)')
plt.xlabel('Date')
plt.ylabel('kW');

This is much smoother and might be easier for the model to handle. Although it will also have less data to draw from. I will try both the 15min and hourly data to compare how they model, even though are ultimate goal is the 15 min AC power generation.

In [ ]:
target_day = target.resample('D').sum()
fig, ax = plt.subplots(figsize=(15,5))
sns.lineplot(target_day)
plt.title('Daily Power Generation (kW) over whole dataset')
plt.xlabel('Date')
plt.ylabel('kW');

It looks like there might be some longer trends present in the generation, possilby due to the weather patterns. We will try to include some features that could capture this in our model.

## Time series decomposition

In [ ]:
result=seasonal_decompose(target, model='additive', period=96*7);
result.plot();

Decomposing our target captures some of the trends a bit better. Of course, the daily pattern is again clear. The residual plot does show quite a bit of noise, which will be challenging for our model to handle. Maybe some of the exogenous features can help with this. 

## Establishing Baseline
We are now ready to establish a baseline. Our baseline will be that the generation was the same as the day before at the same 15 minute interval. We will use root mean squared error (RMSE) scoring throughout and set aside the last 4 days of data to test between models. This will give us 3 days to work with since we are forecasting a day ahead. RMSE is used to give our model an extra penalty for large errors (since the errors are squared in the calculation). The smaller the score the better. 

In [ ]:
day = 96
#last 3 days for testing and the rest for training
target_test = target[-3*day:]
target_train = target[:-3*day]
#moving target back a day
baseline_preds = target.shift(day)[-3*day:]
baseline_score = np.round(mean_squared_error(target_test, baseline_preds, squared=False), 4)
print('baseline_score: ', baseline_score)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
# plt.plot(X[X.index > '2020-06-10'].index, y_pred[X.index > '2020-06-10'], alpha=0.5)
plt.plot(baseline_preds, label='baseline')
plt.plot(target_test);
plt.title(f'Baseline RMSE Score {baseline_score}')
plt.legend(['Baseline preds', 'Actual']);

This is actually not too bad. It does capture some stretches pretty accurately. Likely the day before data will play a prominent role in our model.

## How to handle inverters? / Weather impact
So far we have had all of the inverters in one column of our dataframe with all of the their individual generation data in additional rows. For modeling though we should aggregate our dataframe to one datatime and make that our index. This leaves the question of what to do with the inverters. We could: aggregate accross them, greatly simplifying our dataframe, but possibly losing helpful information; or pivot them as new features to our dataframe making it much more complex; model each inverter seperately and make our model the sum of their predicitons. Also, as per our project scope, let's use this chance to see how the weather data impacts the models performance. We'll setup for each scenerio below

In [ ]:
generation = ['AC_POWER', 'DC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']
weather = ['AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
all_features = ['AC_POWER', 'DC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD', 'AMBIENT_TEMPERATURE', 'MODULE_TEMPERATURE', 'IRRADIATION']
#Aggregating data across inverters. We will sum the generation data and take the mean of the weather data. Also, adding back basic datetime features
df_agg = pd.concat([df.groupby('DATE_TIME')[generation].sum(numeric_only=True), df.groupby('DATE_TIME').mean(numeric_only=True)[weather]], axis=1)

df_agg['HOUR'] = df_agg.index.hour
df_agg['DAY'] = df_agg.index.dayofyear
df_agg['DAY_WEEK'] = df_agg.index.dayofweek
df_agg['MINUTES_15'] = df_agg.index.time

#This maps the 15 minute intervals over the course of the day to ints 1-96. 
df_agg['MINUTES_15'] = df_agg.MINUTES_15.map({df_agg.MINUTES_15.unique()[i-1]:i for i in range(1, 97)})
#Change day of year to day of data
df_agg['DAY'] = df_agg.DAY.map({df_agg.DAY.unique()[i-1]:i for i in range(1, 35)})
#Pivoting inverter generation data as features and adding back datatime features
inverter_dfs = []
for i in inverters:
    df_inverter = df[df.INVERTER == i][['INVERTER', 'DATE_TIME', 'AC_POWER', 'DC_POWER', 'DAILY_YIELD', 'TOTAL_YIELD']]
    df_inverter.pop('INVERTER')
    date_time = df_inverter.pop('DATE_TIME')
    df_inverter.index = date_time
    df_inverter.columns += f'_{i}'
    inverter_dfs.append(df_inverter)
    
df_sep = pd.concat(inverter_dfs, axis=1, ignore_index=False)
df_sep = pd.concat([df_sep, df_agg], axis=1)

all_features_seperated = list(df_sep.columns)

#Without the weather data
df_agg_no_weather = df_agg.drop(weather, axis=1)
df_sep_no_weather = df_sep.drop(weather, axis=1)

#For deep learning model
df_agg_dl = df_agg.copy()
df_sep_dl = df_sep.copy()

## Model Selection
Finally, we have made it to selecting our model. Per the project scope were are going to use an XGBoost algorithm. XGBoost is chosen since it is known to have generally good results and is quick to train. 

### CV
We'll use GridSearchCV to both help tune our model and handle the cross-validation for us. Since we are of course using time series data the CV needs to be sensitive to target leakage by only testing on future indices. Luckily, sklearn has a function to help us. We'll leave off the last 4 days for future use (to have 3 days of day ahead testing). Setting the test size to 96 will have it test one day ahead for a full day at a time. Also, let's have no limit on the train size to make it an expanding training window. This might help the model capture some longer term trends.

In [ ]:
day = 96
tscv = TimeSeriesSplit(test_size=day, n_splits=29)
cv_indices = []
#since all our inverter splits share the same index we'll just choose df_agg
for train_indices, test_indices in tscv.split(df_agg.iloc[:-day*4]):
    cv_indices.append((train_indices, test_indices))

### GridSearch
Now with our CV indices in place are ready to train and test our first models. Let's start with a very basic grid search for our XGBoost regressor and the data aggregated across the inverters. We'll also make a dictionary to store our results.

In [ ]:
results = {}
#making train and test sets for our different inverter splits and with and without the weather data
df_agg_no_weather_train = df_agg_no_weather.iloc[:-day*4]
df_agg_no_weather_test = df_agg_no_weather.iloc[-day*4:-day]

df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]

df_sep_no_weather_train = df_sep_no_weather.iloc[:-day*4]
df_sep_no_weather_test = df_sep_no_weather.iloc[-day*4:-day]

df_sep_train = df_sep.iloc[:-day*4]
df_sep_test = df_sep.iloc[-day*4:-day]

xgb = XGBRegressor(random_state=1)
#param_grid = {'booster': ['dart', 'gbtree', 'gblinear'], 'n_estimators': [50, 100, 150], 'eta': [0.05, 0.1, 0.3]}
param_grid = {}
gs = GridSearchCV(xgb, param_grid=param_grid, cv=cv_indices, verbose=10, scoring='neg_root_mean_squared_error')
gs.fit(df_agg_no_weather_train, target_train)

In [ ]:
print("CV RMSE: ", -np.round(gs.best_score_, 4), "CV RMSE STD: ",
np.round(gs.cv_results_['std_test_score'][0], 4))

In [ ]:
best_xgb = gs.best_estimator_
pd.DataFrame(best_xgb.feature_importances_.T, index=list(best_xgb.feature_names_in_), columns=['Feature_Importance']).sort_values('Feature_Importance', ascending=False)

In [ ]:
agg_no_weather_preds = best_xgb.predict(df_agg_no_weather_test)
agg_no_weather_score = np.round(mean_squared_error(target_test, agg_no_weather_preds, squared=False), 4)
print('RMSE 3-day test period ', agg_no_weather_score)

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
# plt.plot(X[X.index > '2020-06-10'].index, y_pred[X.index > '2020-06-10'], alpha=0.5)
plt.plot(df_agg_test.index, np.array(agg_no_weather_preds), label='agg')
plt.plot(target_test.index, np.array(target_test));
plt.title(f'Agg no weather RMSE Score {agg_no_weather_score}')
plt.legend(['Agg no weather preds', 'Actual']);

Let's streamline this a bit and make this a function since we will be train our model several times.

In [ ]:
def model_train_plot(df_train, df_test, target_train, target_test, name, results_dict ={}, cv_indices = cv_indices, param_grid = {}, estimator=XGBRegressor(random_state=1)):
    results_dict = results_dict
    gs = GridSearchCV(xgb, param_grid=param_grid, cv=cv_indices, scoring='neg_root_mean_squared_error', verbose=0)
    gs.fit(df_train, target_train);
    
    params_tried = gs.param_grid
    best_params = gs.best_params_
    rmse_score_cv = -np.round(gs.best_score_, 4)
    std_score_cv = np.round(gs.cv_results_['std_test_score'][0], 4)

    best_estimator = gs.best_estimator_
    feature_importances_df = pd.DataFrame(best_estimator.feature_importances_.T, index=list(best_estimator.feature_names_in_), columns=['Feature_Importance']).sort_values('Feature_Importance', ascending=False)

    model_preds = best_estimator.predict(df_test)
#     model_preds['AC_POWER'] = model_preds['AC_POWER'].mask(df.HOUR.isin([0]), 0)
    rmse_score_test = np.round(mean_squared_error(target_test, model_preds, squared=False), 4)
    #day_ahead_rmse_score_test = np.round(mean_squared_error(target_test.iloc[:day], model_preds[:day], squared=False), 4)
    

    fig, ax = plt.subplots(figsize=(15,5))
    plt.plot(df_test.index, np.array(model_preds), label='agg')
    plt.plot(target_test.index, np.array(target_test));
    plt.title(f'AC Power {name}_preds vs actual')
    plt.ylabel('AC Power (kW)')
    plt.xlabel('Day')
#     plt.figtext(.13, .85, s=f'Single Day Ahead RMSE Score test set: {day_ahead_rmse_score_test}')
    plt.figtext(.13, .85, s=f'RMSE Score test set: {rmse_score_test}')
    plt.figtext(.13, .82, s=f'CV RMSE Score: {rmse_score_cv}')
    plt.figtext(.13, .79, s=f'CV STD Score: {std_score_cv}')
    plt.figtext(.13, .76, s=f'Best Params: {best_params}')
    plt.legend([f'{name} preds', 'Actual'], loc='upper right');
    results_dict.update({f'{name}':{'best_params':best_params, 'params_tried':params_tried, 'rmse_score_cv': rmse_score_cv, 'std_score_cv':std_score_cv, 'rmse_score_test': rmse_score_test}})
    return fig, gs, results_dict, feature_importances_df.T, model_preds

Now that we have a training funtion in place let's call it with each of the four scenarios. We will also give it a basic param_grid to hopefully give each scenario a fair chance.

In [ ]:
# param_grid = {'eta': [0.05, 0.1, 0.3], 'n_estimators': [50, 100, 150]}
# fig, gs_agg_no_weather, results, feat, preds = model_train_plot(df_agg_no_weather_train, df_agg_no_weather_test, target_train, target_test, 'Aggregated_no_weather_gen', param_grid=param_grid)
# feat

In [ ]:
# fig, gs_agg, results, feat, preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen', param_grid=param_grid)
# feat

In [ ]:
# fig, gs_sep_no_weather, results, feat, pred = model_train_plot(df_sep_no_weather_train, df_sep_no_weather_test, target_train, target_test, 'Seperated_no_weather_gen', param_grid=param_grid)
# feat

In [ ]:
# fig, gs_sep, results, feat, preds = model_train_plot(df_sep_train, df_sep_test, target_train, target_test, 'Seperated_gen', param_grid=param_grid)
# feat

All of them outperformed the baseline on the 3 day test, though not by much. It is close between the model with only the aggregated generation data and the aggregated generation data that also includes the weather. The aggregated model with the weather does seem to do a better job with tracking the more subtle peaks and valleys in our target though.

Seperating the inverters out doesn't seem to help the model. The cause of this is possibly that the extra features overload the model and it has trouble grasping the most important ones. This is evident in the feature importances in models that include all the inverters seperate. It feels a bit arbitrary, which inverters were given the most importance by the model. Also, there doesn't seem to be any improvement on the the day after the model trains with the additional features (single day ahead RMSE test). It will be interesting to see have the inverters seperate is something our deep learning model can make use of later, but for now will just use the aggregated generation data

So let's pick the aggregated generation data with weather data. This model should be a nice starting point and since it is one of the simpler scenarios it will be quicker to train and give us more bandwidth to engineer additional features.

## Feature Engineering
Now that we have an idea of how model is performing with just some basic datetime features, let's see how we can improve the performance with adding additional features.

### SIN/COS features
First we'll make a cosine and sine representation of the day. This technique was taken from this times series <a href="https://www.tensorflow.org/tutorials/structured_data/time_series#time">tutorial</a>. It should help the model pick out the clear daily pattern. 

In [ ]:
#just using the selected dataset that doesn't include weather or seperate generation data for each inverter
timestamp_s = df_agg.index.map(pd.Timestamp.timestamp)
day_s = 24*60*60
df_agg['Day_sin'] = np.sin(timestamp_s  * (2 * np.pi / day_s))
df_agg['Day_cos'] = np.cos(timestamp_s * (2 * np.pi / day_s))

#respliting with new features    
df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]

Let's try our model again with this added feature. We will use the best params from the grid search as well.

In [ ]:
param_grid = {'eta': [0.05], 'n_estimators': [100]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train,  target_test, name='Aggregated_gen w cos/sin feature', param_grid=param_grid)
feat

Already a fair bit of improvement and it is by far the most important feature! The cosine tracks much better with our target (see below). Let's generalize our sin/cos and make some new features with different frequencies. Perhaps we can pick up on some additional trends/patterns in our model.

In [ ]:
fig = plt.subplot()
sns.lineplot(df_agg['Day_cos'].iloc[:4*day])
sns.lineplot(df_agg['Day_sin'].iloc[:4*day])
sns.lineplot((df_agg['AC_POWER'].iloc[:4*day] - df_agg['AC_POWER'].mean())/ df_agg['AC_POWER'].std())
fig.set_ylabel('')
fig.set_xlabel('')
fig.set_yticklabels('')
fig.set_xticklabels('')
fig.legend(['Cos'])
plt.title('Cos/Sin/AC Power');


This shows how cosine does a better job of tracking with the AC power. Let's remove the sin feature. We can also make the cosine negative to better align with the shape of the power generation.

In [ ]:
df_agg = df_agg.drop('Day_sin', axis=1)
df_agg['Day_cos'] = -df_agg.Day_cos

#respliting with new features    
df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]

#retraining 
param_grid = {'eta': [0.05], 'n_estimators': [100]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen w/ cos feature', param_grid=param_grid)
feat


It is a bit better. Let's generalize the cos/sin features some and add some more frequencies to see if we can pick up on additional trends.

In [ ]:
def day_cos_sin(df, freq=1, type='both'):
    timestamp_s = df.index.map(pd.Timestamp.timestamp)
    day_s = 24*60*60
    if type=='both':
        df[f'Day_sin_{freq}'] = np.sin(timestamp_s  * (2 * freq * np.pi / day_s))
        df[f'Day_cos_{freq}'] = np.cos(timestamp_s * (2 * freq * np.pi / day_s))
    elif type=='cos':
        df[f'Day_cos_{freq}'] = np.cos(timestamp_s * (2 * freq * np.pi / day_s))
    elif type=='sin':
        df[f'Day_sin_{freq}'] = np.sin(timestamp_s  * (2 * freq * np.pi / day_s))
    return df

#We will just add cos features and go up to a 'week'
df_agg = day_cos_sin(df_agg, .5, type='cos')
df_agg = day_cos_sin(df_agg, 2, type='cos')
df_agg = day_cos_sin(df_agg, 3, type='cos')
df_agg = day_cos_sin(df_agg, 4, type='cos')
df_agg = day_cos_sin(df_agg, 5, type='cos')
df_agg = day_cos_sin(df_agg, 6, type='cos')
df_agg = day_cos_sin(df_agg, 7, type='cos')

#respliting with new features    
df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]

In [ ]:
param_grid = {'eta': [0.05], 'n_estimators': [100]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen w/ cos features ', param_grid=param_grid)
feat

The cosine features over 1-4 days seem somewhat helpful, but not as much after that. This suggests that our data have useful information up to a period of 4 days. Let's remove the cosine features beyond 4 days and the half-day feature of 0.5.

In [ ]:
df_agg = df_agg.drop(['Day_cos_0.5', 'Day_cos_5', 'Day_cos_6', 'Day_cos_7'], axis=1)

#respliting with new features    
df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]

param_grid = {'eta': [0.05], 'n_estimators': [100]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen w cos up to 4 days', param_grid=param_grid)
feat

### Lag and rolling features

Now will add a few more kinds of features. We'll focus these in on the generation data (really no reason to add them to the datetime features). Let's start with a lag feature. Since our cosine features were picking up some potentially useful information up to a 4 day period, we'll make a lag feature going back 4 days. This will introduce some nulls for these features at the beginning of the dataset as we shift the lag features forward, but our model should be able to handle them (see xgboost FAQ: https://xgboost.readthedocs.io/en/stable/faq.html).

In [ ]:
for f in all_features:
    new_features = [df_agg]
    for i in range(day, day*4, day):
        lag_feature= df_agg[f].shift(i)
        lag_feature.name = f'lag_{f}{i}'
        new_features.append(lag_feature)
    df_agg = pd.concat(new_features, axis=1)
    
#respliting with new features    
df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]
    
#retraining with new features
param_grid = {'eta': [0.05], 'n_estimators': [100]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, name='Aggregated_gen w/ lag features', param_grid=param_grid)
feat


This does seem to help some overall. Now we'll add some rolling features. This should give our model a smoother picture of the previous generation data to work with. Similiar to when we resampled the target above to hourly and it removed much of the noise.

In [ ]:
for f in all_features:
    new_features = [df_agg]
    lag_feature= df_agg[f].rolling(2).mean()
    lag_feature.name = f'rolling_{f}'
    new_features.append(lag_feature)
    df_agg = pd.concat(new_features, axis=1)

for f in all_features:
    new_features = [df_agg]
    lag_feature= df_agg[f].shift(day).rolling(5, center=True).mean()
    lag_feature.name = f'rolling_{f}_day'
    new_features.append(lag_feature)
    df_agg = pd.concat(new_features, axis=1)

for f in all_features:
    new_features = [df_agg]
    lag_feature= df_agg[f].shift(2*day).rolling(5, center=True).mean()
    lag_feature.name = f'rolling_{f}_day_2'
    new_features.append(lag_feature)
    df_agg = pd.concat(new_features, axis=1)
    
for f in all_features:
    new_features = [df_agg]
    lag_feature= df_agg[f].shift(3*day).rolling(5, center=True).mean()
    lag_feature.name = f'rolling_{f}_day_3'
    new_features.append(lag_feature)
    df_agg = pd.concat(new_features, axis=1)
    
#respliting with new features    
df_agg_train = df_agg.iloc[:-day*4]
df_agg_test = df_agg.iloc[-day*4:-day]

#retraining with new features
param_grid = {'eta': [0.05], 'n_estimators': [100]}
fig, gs_agg, results, feat, model_preds= model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen w more features', param_grid=param_grid)
feat


Perhaps some performance gains, but it is hard to tell for sure. We might have too many features now and/or need to retune the parameters to account for the additional features the model has to process. Let's start with retuning the parameters

## Tuning parameters

Let's do a couple rounds of gridsearch including some additional parameters to see if we can improve the performance

In [ ]:
#1st params tried param_grid = {'booster' : ['dart', 'gbtree'], 'n_estimators': [50, 100, 150], 'eta': [0.01, 0.5, 0.1] }
#best params {'booster' : ['gbtree'], 'n_estimators': [50], 'eta': [0.1] }
# Additional parameters
# 2nd param_grid tried = {'booster' : ['gbtree'], 'max_depth': [3, 5, 10], 'colsample_bytree': [0.5, 0.75, 1], 'subsample': [0.6, 0.8, 1], 'n_estimators' : [50], 'eta' : [0.1]}
#best params {'booster' : ['gbtree'], 'max_depth':  [3], 'colsample_bytree': [1], 'subsample': [0.8]}
#3rd param_grid tried param_grid = {'booster' : ['gbtree'], 'max_depth': [1, 2, 3], 'colsample_bytree': [1], 'subsample': [0.8], 'n_estimators': [50], 'eta': [0.1]}
param_grid = {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.8], 'n_estimators': [50], 'eta': [0.1]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen tune', param_grid=param_grid)
feat

There is a pretty big gain in performance with the max depth set to 1. The model becomes a fair bit simpler and it is less likely to find every peak or valley, but on the average does a decent job. Depending on the exact application this could be underfitting, but since it works well with our chosen metric we'll stick with it.

### Feature selection with Random Forest

To simplfy our model even further and see if we can help it focus on the more important features, let's use a random forest model to filter out some of the less helpful features.

In [ ]:
rfc = RandomForestRegressor(random_state=1)
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen random forest', estimator=rfc, param_grid={})
feat

In [ ]:
model = SelectFromModel(gs_agg.best_estimator_, prefit=True, threshold=0.001)

df_agg_new = pd.DataFrame(model.transform(df_agg), columns=model.get_feature_names_out(df_agg.columns), index=df_agg.index)

df_agg_train = df_agg_new.iloc[:-day*4]
df_agg_test = df_agg_new.iloc[-day*4:-day]

param_grid = {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.8], 'n_estimators': [50], 'eta': [0.1]}
fig, gs_agg, results, feat, model_preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen feature selection', param_grid=param_grid)
feat

### Final tuning
Other than the Day_cos feature the rolling features ended up being the most useful. Since we went with a simpler model that greatly smooths the data over the course of the day, it makes perfect sense that these rolling features played a prominent role. One last fine tuning of params before wrapping up or XGBoost model and moving to the LSTM one.

In [ ]:
#1st param_grid tried= {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.7, 0.8, 0.9], 'n_estimators': [25, 50, 75], 'eta': [0.75, 0.1, 0.2]}
#best param_grid on 1st try {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.7], 'n_estimators': [50], 'eta': [0.1]}
#2nd param_grid tried = {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.65, 0.7, 0.75], 'n_estimators': [45, 50, 55], 'eta': [0.095, 0.1, 0.15]}
#best param_grid on 2nd try {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.65], 'n_estimators': [45], 'eta': [0.1]}
#3rd param_grid tried {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.65], 'n_estimators': [42, 43, 44, 45, 46], 'eta': [0.099,0.1,0.11]}
#best param grid {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.64, 0.65, 0.66], 'n_estimators': [42], 'eta': [0.11]}
#last try {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.64, 0.65, 0.66], 'n_estimators': [38, 39, 40, 41, 42], 'eta': [0.11, 0.12]}
#best param below
param_grid = {'booster' : ['gbtree'], 'max_depth': [1], 'colsample_bytree': [1], 'subsample': [0.65], 'n_estimators': [42], 'eta': [0.11]}
fig, gs_agg, results, feat, preds = model_train_plot(df_agg_train, df_agg_test, target_train, target_test, 'Aggregated_gen final tune', param_grid=param_grid)
feat

## Onto Deep learning

Now that we have completed our XGBoost model, let's move to our deep learning model. For our deep learning model we will use a LSTM layer to make a recurrent model (RNN). This should allow our model to store helpful information about that past to use for its forecast. It will take in the data from the last 4 days and generate a forecast of AC Power for each 15 minute interval for the next day. 

Most of the preprocessing has already been finished before, but we additionally need to normalize the input data. Also, we'll go back check to see how it performs using the inverters seperated and aggregated. We'll include the weather data in both. My guess is that our deep learning model might be able to make better use of the additional features and show some improvement with the inverters seperated. First, we have to setup the data to feed into our deep learning model.

In [ ]:
#removing features (besides basic datetime ones)
df_agg = df_agg_dl
df_sep = df_sep_dl
day = 96
tscv = TimeSeriesSplit(test_size=day, n_splits=29, max_train_size=4*day)
cv_indices = []
#since all our inverter splits share the same index we'll just choose df_agg
for train_indices, test_indices in tscv.split(df_agg.iloc[:-day*4]):
    cv_indices.append((train_indices, test_indices))

In [ ]:
### df_sep full data including weather

inputs_sep_trains = []
labels_sep_trains = []
for i in range(3, len(cv_indices)-4):
    input_sep_train = (df_sep.iloc[cv_indices[i][0]] - df_sep.iloc[cv_indices[i][0]].mean()) / df_sep.iloc[cv_indices[i][0]].std()
    label_sep_train = (target.iloc[cv_indices[i][1]] - target.iloc[cv_indices[i][1]].mean()) / target.iloc[cv_indices[i][1]].std()
    label_sep_train = pd.DataFrame(label_sep_train, columns=['AC_POWER'])
    inputs_sep_trains.append(input_sep_train)
    labels_sep_trains.append(label_sep_train)
    
inputs_sep_train_stack = tf.stack(inputs_sep_trains)
labels_sep_train_stack = tf.stack(labels_sep_trains)

inputs_sep_vals = []
labels_sep_vals = []
for i in range(len(cv_indices)-4 , len(cv_indices)):
    input_sep_val = (df_sep.iloc[cv_indices[i][0]] - df_sep.iloc[cv_indices[i][0]].mean()) / df_sep.iloc[cv_indices[i][0]].std()
    label_sep_val = (target.iloc[cv_indices[i][1]] - target.iloc[cv_indices[i][1]].mean()) / target.iloc[cv_indices[i][1]].std()
    label_sep_val = pd.DataFrame(label_sep_val, columns=['AC_POWER'])
    inputs_sep_vals.append(input_sep_val)
    labels_sep_vals.append(label_sep_val)
    
inputs_sep_val_stack = tf.stack(inputs_sep_vals)
labels_sep_val_stack = tf.stack(labels_sep_vals)
    
val = (inputs_sep_val_stack, labels_sep_val_stack)
 

In [ ]:
### Multi output sep
MAX_EPOCHS = 2000

model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(10, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(96*1,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([96, 1])
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=20,
                                                    mode='min',
                                                    restore_best_weights=True)

model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(x=inputs_sep_train_stack, y=labels_sep_train_stack, epochs=MAX_EPOCHS,
                    validation_data=val,
                    callbacks=[early_stopping],
                    verbose=0);

In [ ]:
day_1_labels = target.iloc[-day*3:-day*2]
day_2_labels = target.iloc[-day*2:-day*1]
day_3_labels = target.iloc[-day:]

test_labels = [day_1_labels, day_2_labels, day_3_labels]
test_labels_stack = tf.stack(test_labels)

day_1_inputs = (df_sep.iloc[-day*7:-day*3] - df_sep.iloc[-day*7:-day*3].mean()) / df_sep.iloc[-day*7:-day*3].std()
day_2_inputs = (df_sep.iloc[-day*6:-day*2] - df_sep.iloc[-day*6:-day*2].mean()) / df_sep.iloc[-day*6:-day*2].std()
day_3_inputs = (df_sep.iloc[-day*5:-day] - df_sep.iloc[-day*5:-day].mean()) / df_sep.iloc[-day*5:-day].std()

test_inputs = [day_1_inputs, day_2_inputs, day_3_inputs]
test_inputs_stack = tf.stack(test_inputs)

target_norm_preds = model.predict(test_inputs_stack)

labels_preds_day_1 = target_norm_preds[0] * df_sep.iloc[-day*7:-day*3].AC_POWER.std() + df_sep.iloc[-day*7:-day*3].AC_POWER.mean()
labels_preds_day_2 = target_norm_preds[0] * df_sep.iloc[-day*6:-day*2].AC_POWER.std() + df_sep.iloc[-day*6:-day*2].AC_POWER.mean()
labels_preds_day_3 = target_norm_preds[0] * df_sep.iloc[-day*5:-day].AC_POWER.std() + df_sep.iloc[-day*5:-day].AC_POWER.mean()
test_labels_stack[0]

fig, ax = plt.subplots(figsize=(15,5))
plt.plot(labels_preds_day_1)
plt.plot(test_labels_stack[0])

In [ ]:
day_1_error = mean_squared_error(test_labels_stack[0], labels_preds_day_1, squared=False)
day_2_error = mean_squared_error(test_labels_stack[1], labels_preds_day_2, squared=False)
day_3_error = mean_squared_error(test_labels_stack[2], labels_preds_day_3, squared=False)
lstm_rsme_score_test_set = np.round((day_1_error + day_2_error + day_3_error) / 3, 4)
print("LSTM RSME test set:" , lstm_rsme_score_test_set)

In [ ]:
### df_agg full data including weather
inputs_agg_trains = []
labels_agg_trains = []
for i in range(3, len(cv_indices)-4):
    input_agg_train = (df_agg.iloc[cv_indices[i][0]] - df_agg.iloc[cv_indices[i][0]].mean()) / df_agg.iloc[cv_indices[i][0]].std()
    label_agg_train = (target.iloc[cv_indices[i][1]] - target.iloc[cv_indices[i][1]].mean()) / target.iloc[cv_indices[i][1]].std()
    label_agg_train = pd.DataFrame(label_sep_train, columns=['AC_POWER'])
    inputs_agg_trains.append(input_agg_train)
    labels_agg_trains.append(label_agg_train)
    
inputs_agg_train_stack = tf.stack(inputs_agg_trains)
labels_agg_train_stack = tf.stack(labels_agg_trains)

inputs_agg_vals = []
labels_agg_vals = []
for i in range(len(cv_indices)-4 , len(cv_indices)):
    input_agg_val = (df_agg.iloc[cv_indices[i][0]] - df_agg.iloc[cv_indices[i][0]].mean()) / df_agg.iloc[cv_indices[i][0]].std()
    label_agg_val = (target.iloc[cv_indices[i][1]] - target.iloc[cv_indices[i][1]].mean()) / target.iloc[cv_indices[i][1]].std()
    label_agg_val = pd.DataFrame(label_sep_val, columns=['AC_POWER'])
    inputs_agg_vals.append(input_agg_val)
    labels_agg_vals.append(label_agg_val)
    
inputs_agg_val_stack = tf.stack(inputs_agg_vals)
labels_agg_val_stack = tf.stack(labels_agg_vals)
    
val = (inputs_agg_val_stack, labels_agg_val_stack)

In [ ]:
### Multi output sep
MAX_EPOCHS = 2000

model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(10, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(96*1,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([96, 1])
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=20,
                                                    mode='min',
                                                    restore_best_weights=True)

model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(x=inputs_agg_train_stack, y=labels_agg_train_stack, epochs=MAX_EPOCHS,
                    validation_data=val,
                    callbacks=[early_stopping],
                    verbose=0);

In [ ]:
day_1_labels = target.iloc[-day*3:-day*2]
day_2_labels = target.iloc[-day*2:-day*1]
day_3_labels = target.iloc[-day:]

test_labels = [day_1_labels, day_2_labels, day_3_labels]
test_labels_stack = tf.stack(test_labels)

day_1_inputs = (df_agg.iloc[-day*7:-day*3] - df_agg.iloc[-day*7:-day*3].mean()) / df_agg.iloc[-day*7:-day*3].std()
day_2_inputs = (df_agg.iloc[-day*6:-day*2] - df_agg.iloc[-day*6:-day*2].mean()) / df_agg.iloc[-day*6:-day*2].std()
day_3_inputs = (df_agg.iloc[-day*5:-day] - df_agg.iloc[-day*5:-day].mean()) / df_agg.iloc[-day*5:-day].std()

test_inputs = [day_1_inputs, day_2_inputs, day_3_inputs]
test_inputs_stack = tf.stack(test_inputs)

target_norm_preds = model.predict(test_inputs_stack)

labels_preds_day_1 = target_norm_preds[0] * df_agg.iloc[-day*7:-day*3].AC_POWER.std() + df_agg.iloc[-day*7:-day*3].AC_POWER.mean()
labels_preds_day_2 = target_norm_preds[0] * df_agg.iloc[-day*6:-day*2].AC_POWER.std() + df_agg.iloc[-day*6:-day*2].AC_POWER.mean()
labels_preds_day_3 = target_norm_preds[0] * df_agg.iloc[-day*5:-day].AC_POWER.std() + df_agg.iloc[-day*5:-day].AC_POWER.mean()
test_labels_stack[0]

fig, ax = plt.subplots(figsize=(15,5))
plt.plot(labels_preds_day_1)
plt.plot(test_labels_stack[0])
plt.legend(['predicted', 'actual'])

In [ ]:
day_1_error = mean_squared_error(test_labels_stack[0], labels_preds_day_1, squared=False)
day_2_error = mean_squared_error(test_labels_stack[1], labels_preds_day_2, squared=False)
day_3_error = mean_squared_error(test_labels_stack[2], labels_preds_day_3, squared=False)
lstm_rsme_score_test_set = np.round((day_1_error + day_2_error + day_3_error) / 3, 4)
print("LSTM RSME test set aggregated:" , lstm_rsme_score_test_set)

The model with the inverters seperated is the clear winner in this. That shows as predicted that our deep learning model is able to do more with complex data. With this in mind, let's try to add all of the features we created during our previous feature engineering. Since this will include the seperate inverter features it could well overload it, but it will be interesting to see.

In [ ]:
df_sep = df_sep_dl

df_sep = day_cos_sin(df_sep, 1, type='cos')
df_sep = day_cos_sin(df_sep, 2, type='cos')
df_sep = day_cos_sin(df_sep, 3, type='cos')
df_sep = day_cos_sin(df_sep, 4, type='cos')

for f in all_features_seperated:
    new_features = [df_sep]
    for i in range(day, day*4, day):
        lag_feature= df_sep[f].shift(i)
        lag_feature.name = f'lag_{f}{i}'
        new_features.append(lag_feature)
    df_sep = pd.concat(new_features, axis=1)
    
for f in all_features_seperated:
    new_features = [df_sep]
    lag_feature= df_sep[f].rolling(2).mean()
    lag_feature.name = f'rolling_{f}'
    new_features.append(lag_feature)
    df_sep = pd.concat(new_features, axis=1)

for f in all_features_seperated:
    new_features = [df_sep]
    lag_feature= df_sep[f].shift(day).rolling(5, center=True).mean()
    lag_feature.name = f'rolling_{f}_day'
    new_features.append(lag_feature)
    df_sep = pd.concat(new_features, axis=1)

for f in all_features_seperated:
    new_features = [df_sep]
    lag_feature= df_sep[f].shift(2*day).rolling(5, center=True).mean()
    lag_feature.name = f'rolling_{f}_day_2'
    new_features.append(lag_feature)
    df_sep = pd.concat(new_features, axis=1)
    
for f in all_features_seperated:
    new_features = [df_sep]
    lag_feature= df_sep[f].shift(3*day).rolling(5, center=True).mean()
    lag_feature.name = f'rolling_{f}_day_3'
    new_features.append(lag_feature)
    df_sep = pd.concat(new_features, axis=1)
    
imputer = KNNImputer(n_neighbors=10)
df_sep = pd.DataFrame(imputer.fit_transform(df_sep), columns=df_sep.columns, index=df_sep.index)
df_sep.info()

In [ ]:
### df_sep full data including all created features
inputs_sep_full_trains = []
labels_sep_full_trains = []
for i in range(4, len(cv_indices)-4):
    input_sep_full_train = (df_sep.iloc[cv_indices[i][0]] - df_sep.iloc[cv_indices[i][0]].mean()) / df_sep.iloc[cv_indices[i][0]].std()
    label_sep_full_train = (target.iloc[cv_indices[i][1]] - target.iloc[cv_indices[i][1]].mean()) / target.iloc[cv_indices[i][1]].std()
    label_sep_full_train = pd.DataFrame(label_sep_full_train, columns=['AC_POWER'])
    inputs_sep_full_trains.append(input_sep_full_train)
    labels_sep_full_trains.append(label_sep_full_train)
    
inputs_sep_full_train_stack = tf.stack(inputs_sep_full_trains)
labels_sep_full_train_stack = tf.stack(labels_sep_full_trains)

inputs_sep_full_vals = []
labels_sep_full_vals = []
for i in range(len(cv_indices)-4 , len(cv_indices)):
    input_sep_full_val = (df_sep.iloc[cv_indices[i][0]] - df_sep.iloc[cv_indices[i][0]].mean()) / df_sep.iloc[cv_indices[i][0]].std()
    label_sep_full_val = (target.iloc[cv_indices[i][1]] - target.iloc[cv_indices[i][1]].mean()) / target.iloc[cv_indices[i][1]].std()
    label_sep_full_val = pd.DataFrame(label_sep_full_val, columns=['AC_POWER'])
    inputs_sep_full_vals.append(input_sep_full_val)
    labels_sep_full_vals.append(label_sep_full_val)
    
inputs_sep_full_val_stack = tf.stack(inputs_sep_full_vals)
labels_sep_full_val_stack = tf.stack(labels_sep_full_vals)
    
val_full = (inputs_sep_full_val_stack, labels_sep_full_val_stack)
 

In [ ]:
### Multi output sep
MAX_EPOCHS = 2000

model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.LSTM(20, return_sequences=False),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(96*1,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([96, 1])
])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=20,
                                                    mode='min',
                                                    restore_best_weights=True)

model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

history = model.fit(x=inputs_sep_full_train_stack, y=labels_sep_full_train_stack, epochs=MAX_EPOCHS,
                    validation_data=val_full,
                    callbacks=[early_stopping],
                    verbose=2);

In [ ]:
day_1_labels = target.iloc[-day*3:-day*2]
day_2_labels = target.iloc[-day*2:-day*1]
day_3_labels = target.iloc[-day:]

test_labels = [day_1_labels, day_2_labels, day_3_labels]
test_labels_stack = tf.stack(test_labels)

day_1_inputs = (df_sep.iloc[-day*7:-day*3] - df_sep.iloc[-day*7:-day*3].mean()) / df_sep.iloc[-day*7:-day*3].std()
day_2_inputs = (df_sep.iloc[-day*6:-day*2] - df_sep.iloc[-day*6:-day*2].mean()) / df_sep.iloc[-day*6:-day*2].std()
day_3_inputs = (df_sep.iloc[-day*5:-day] - df_sep.iloc[-day*5:-day].mean()) / df_sep.iloc[-day*5:-day].std()

test_inputs = [day_1_inputs, day_2_inputs, day_3_inputs]
test_inputs_stack = tf.stack(test_inputs)

target_norm_preds = model.predict(test_inputs_stack)

labels_preds_day_1 = target_norm_preds[0] * df_sep.iloc[-day*7:-day*3].AC_POWER.std() + df_sep.iloc[-day*7:-day*3].AC_POWER.mean()
labels_preds_day_2 = target_norm_preds[0] * df_sep.iloc[-day*6:-day*2].AC_POWER.std() + df_sep.iloc[-day*6:-day*2].AC_POWER.mean()
labels_preds_day_3 = target_norm_preds[0] * df_sep.iloc[-day*5:-day].AC_POWER.std() + df_sep.iloc[-day*5:-day].AC_POWER.mean()
test_labels_stack[0]

fig, ax = plt.subplots(figsize=(15,5))
plt.plot(labels_preds_day_1)
plt.plot(test_labels_stack[0])

In [ ]:
day_1_error = mean_squared_error(test_labels_stack[0], labels_preds_day_1, squared=False)
day_2_error = mean_squared_error(test_labels_stack[1], labels_preds_day_2, squared=False)
day_3_error = mean_squared_error(test_labels_stack[2], labels_preds_day_3, squared=False)
lstm_rsme_score_test_set = np.round((day_1_error + day_2_error + day_3_error) / 3, 4)
print("LSTM RSME test set:" , lstm_rsme_score_test_set)

## Wait, wasn't there another plant?
Yes, let's not forget there is a whole other solar plant and associated weather data to model. We'll take our most successful model so far and make a full pipeline out of to process the 2nd solar plant data and make a day ahead forecast for it

## Pipeline

To make a full pipeline we can 

## An easier way?

## Conclusions/ Project extensions

# Appendix